In [507]:
! pip install --upgrade pandas

Defaulting to user installation because normal site-packages is not writeable


In [508]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn import ensemble, tree, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor
np.random.seed(453111) # set the random seed

In [509]:
test_id =pd.read_csv("../data/test_id.csv", header=None)
Ames = pd.read_csv("../data/Ames_data.csv")
Ames_Location = Ames[["PID", "Latitude", "Longitude"]]
Ames_Location

,PID,Latitude,Longitude
0,526301100,42.054035,-93.619754
1,526350040,42.053014,-93.619756
2,526351010,42.052659,-93.619387
3,526353030,42.051245,-93.617320
4,527105010,42.060899,-93.638933
...,...,...,...
2925,923275080,41.988964,-93.604776
2926,923276100,41.988314,-93.602680
2927,923400125,41.986510,-93.606847
2928,924100070,41.990921,-93.600190


In [510]:
Ames.drop(['Garage_Yr_Blt',"Condition_2","Utilities","Roof_Matl","Latitude", "Longitude"], axis=1, inplace=True)
Ames.dropna(inplace=True)
Ames.head()

,PID,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Lot_Config,...,Pool_Area,Pool_QC,Fence,Misc_Feature,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Sale_Price
2,526351010,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81,14267,Pave,No_Alley_Access,Slightly_Irregular,Lvl,Corner,...,0,No_Pool,No_Fence,Gar2,12500,6,2010,WD,Normal,172000
23,527402200,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,0,11241,Pave,No_Alley_Access,Slightly_Irregular,Lvl,CulDSac,...,0,No_Pool,No_Fence,Shed,700,3,2010,WD,Normal,149000
114,534152100,Two_Story_1946_and_Newer,Residential_Low_Density,80,10421,Pave,No_Alley_Access,Regular,Lvl,Inside,...,0,No_Pool,Minimum_Privacy,Shed,500,3,2010,WD,Normal,196500
134,535101020,Split_Foyer,Residential_Low_Density,75,11380,Pave,No_Alley_Access,Slightly_Irregular,Lvl,Inside,...,0,No_Pool,No_Fence,Shed,1500,1,2010,WD,Normal,178000
139,535152050,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,70,8197,Pave,No_Alley_Access,Regular,Lvl,Inside,...,0,No_Pool,Minimum_Privacy,Shed,300,2,2010,WD,Normal,214000


In [ ]:
categorical = [var for var in Ames.columns if Ames[var].dtype=='O']
numerical = [var for var in Ames.columns if Ames[var].dtype!='O']
ordinal=["Overall_Qual", "Overall_Cond", "Exter_Qual", "Exter_Cond","Bsmt_Qual","Bsmt_Cond", "Heating_QC","Kitchen_Qual","Fireplace_Qu","Garage_Qual","Garage_Cond",
        "Pool_QC","Lot_Shape","Land_Slope","Bsmt_Exposure","BsmtFin_Type_1","BsmtFin_Type_2","Electrical","Functional",
    "Paved_Drive","Fence"]
nominal =[col for col in categorical if col not in ordinal]
out=Ames.Gr_Liv_Area.sort_values(ascending=False)[:3].index
Ames.drop(out, axis=0, inplace=True)
out=Ames.Total_Bsmt_SF.sort_values(ascending=False)[:3].index
Ames.drop(out, axis=0, inplace=True)

discrete = []
continous=[]
for var in numerical:
    if len(Ames[var].unique())<15:
        discrete.append(var)
    else:
        continous.append(var)
discrete.append("Year_Built")
discrete.append("Year_Remod_Add")
discrete.remove("BsmtFin_SF_1")
continous.remove("Year_Built")
continous.remove("Year_Remod_Add")
continous.append("BsmtFin_SF_1")

In [ ]:
Ames.replace({"Bsmt_Cond" : {"No_Basement" : 0, "Poor" : 1, "Fair" : 2, "Typical": 3, "Good" : 4, "Excellent" : 5},
                    "Bsmt_Exposure" : {"No_Basement":0,"No" : 1, "Mn" : 2, "Av": 3, "Gd" : 4},
                    "BsmtFin_Type_1" : {"No_Basement" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                        "ALQ" : 5, "GLQ" : 6},
                    "BsmtFin_Type_2" : {"No_Basement" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                        "ALQ" : 5, "GLQ" : 6},
                    "Alley": {"No_Alley_Access":0, "Gravel":1, "Paved":2},
                    "Street": {"Grvl":0, "Pave":1},
                    "Central_Air":{"Y":1, "N":0}, 
                    "Land_Contour": {"Bnk":0,"Lvl":1,"HLS":2, "Low":3},
                    "Bldg_Type":{"OneFam":0,"Duplex":1,"TwnhsE":2,"Twnhs":3,"TwoFmCon":4},
                    "Bsmt_Qual" : {"No_Basement" : 0, "Poor" : 1, "Fair" : 2, "Typical": 3, "Good" : 4, "Excellent" : 5},
                    "Exter_Cond" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                    "Exter_Qual" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                    "Fireplace_Qu" : {"No_Fireplace" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                    "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                    "Min2" : 6, "Min1" : 7, "Typ" : 8},
                                            "Garage_Cond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                    "Garage_Qual" : {"No_Garage" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                    "Heating_QC" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                    "Garage_Finish":{"No_Garage":0, "Unf":1, "RFn":2, "Fin":3},
                    "Kitchen_Qual" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                    "Land_Slope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                    "Lot_Shape" : {"Irregular" : 1, "Moderately_Irregular" : 2, "Slightly_Irregular" : 3, "Regular" : 4},
                    "Paved_Drive" : {"Dirt_Gravel" : 0, "Paved" : 1, "Partial_Pavement" : 2},
                    "Pool_QC" : {"No_Pool" : 0, "Fair" : 1, "Typical" : 2, "Good" : 3, "Excellent" : 4},
                    "Overall_Qual":{"Very_Excellent": 10, "Excellent":9,"Very_Good":8, "Good":7, "Above_Average":6, "Average": 5, "Below_Average":4, "Fair": 3,"Poor":2, "Very_Poor":1},
                    "Overall_Cond" :{"Very_Excellent": 10, "Excellent":9,"Very_Good":8, "Good":7, "Above_Average":6, "Average": 5, "Below_Average":4, "Fair": 3,"Poor":2, "Very_Poor":1},
                    "Electrical ": {"Unknown": 0,"SBrkr ": 1,"FuseA": 2, "FuseF": 3, "FuseP": 4,"Mix": 5 },
                    "Fence":{"No_Fence":0, "Minimum_Wood_Wire":1,"Good_Wood":2,"Minimum_Privacy":3,"Good_Privacy":4},
                    "Garage_Cond" : {"No_Garage" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                    }, inplace=True)

Ames["Electrical"].replace({"Unknown": 0,"SBrkr": 1,"FuseA": 2, "FuseF": 3, "FuseP": 4,"Mix": 5 }, inplace=True)

In [ ]:
Age_range = Ames['Year_Built'].max() - Ames['Year_Built'].min()
min_value = int(np.floor(Ames['Year_Built'].min()))
max_value = int(np.ceil(Ames['Year_Built'].max()))

# let's round the bin width
inter_value = int(np.round(Age_range/9))

intervals = [i for i in range(min_value, max_value+inter_value, inter_value)]
labels=[]
for i in range(1, len(intervals)):
    labels.append(i)

In [ ]:
Ames['Year_Built'] = pd.cut(x = Ames['Year_Built'], bins=intervals, labels=labels, include_lowest=True)
Ames['Year_Built']=Ames['Year_Built'].astype('int64', copy=False,errors="ignore")


In [ ]:
Age_range = Ames['Year_Remod_Add'].max() - Ames['Year_Remod_Add'].min()
min_value = int(np.floor(Ames['Year_Remod_Add'].min()))
max_value = int(np.ceil(Ames['Year_Remod_Add'].max()))

In [ ]:
# let's round the bin width
inter_value = int(np.round(Age_range/5))

intervals = [i for i in range(min_value, max_value+inter_value, inter_value)]
labels=[]
for i in range(1, len(intervals)):
    labels.append(i)


In [ ]:
Ames['Year_Remod_Add'] = pd.cut(x = Ames['Year_Remod_Add'], bins=intervals, labels=labels, include_lowest=True)
Ames['Year_Remod_Add']=Ames['Year_Remod_Add'].astype('int64', copy=False,errors="ignore")


In [ ]:
# MS_Subclass
top_5 = [x for x in Ames['MS_SubClass'].value_counts().sort_values(ascending=False).head(5).index]
for label in top_5:
    Ames[label] = np.where(Ames['MS_SubClass']==label, 1, 0)

Ames.drop("MS_SubClass", axis=1, inplace=True)
nominal.remove("MS_SubClass")


In [ ]:
# MS_Zoning
top_3 = [x for x in Ames['MS_Zoning'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['MS_Zoning']==label, 1, 0)

Ames.drop("MS_Zoning", axis=1, inplace=True)
nominal.remove("MS_Zoning")


In [ ]:
# Lot_Config
top_3 = [x for x in Ames['Lot_Config'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['Lot_Config']==label, 1, 0)

Ames.drop("Lot_Config", axis=1, inplace=True)

nominal.remove("Lot_Config")

In [ ]:
# Neighborhood 
top_8 = [x for x in Ames['Neighborhood'].value_counts().sort_values(ascending=False).head(8).index]
for label in top_8:
    Ames[label] = np.where(Ames['Neighborhood']==label, 1, 0)

Ames.drop("Neighborhood", axis=1, inplace=True)
nominal.remove("Neighborhood")

In [ ]:
# Condition_1
top_3 = [x for x in Ames['Condition_1'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['Condition_1']==label, 1, 0)

Ames.drop("Condition_1", axis=1, inplace=True)
nominal.remove("Condition_1")


In [ ]:
# House_Style 
top_3 = [x for x in Ames['House_Style'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['House_Style']==label, 1, 0)

Ames.drop("House_Style", axis=1, inplace=True)
nominal.remove("House_Style")

In [ ]:
# Roof_Style
top_2 = [x for x in Ames['Roof_Style'].value_counts().sort_values(ascending=False).head(2).index]
for label in top_2:
    Ames[label] = np.where(Ames['Roof_Style']==label, 1, 0)

Ames.drop("Roof_Style", axis=1, inplace=True)
nominal.remove("Roof_Style")

In [ ]:
# Exterior_1st 
top_5 = [x for x in Ames['Exterior_1st'].value_counts().sort_values(ascending=False).head(5).index]
for label in top_5:
    Ames[label] = np.where(Ames['Exterior_1st']==label, 1, 0)

Ames.drop("Exterior_1st", axis=1, inplace=True)
nominal.remove("Exterior_1st")

In [ ]:
# Exterior_2nd 
top_5 = [x for x in Ames['Exterior_2nd'].value_counts().sort_values(ascending=False).head(5).index]
for label in top_5:
    Ames[label] = np.where(Ames['Exterior_2nd']==label, 1, 0)

Ames.drop("Exterior_2nd", axis=1, inplace=True)
nominal.remove("Exterior_2nd")

In [ ]:
# Mas_Vnr_Type
top_3 = [x for x in Ames['Mas_Vnr_Type'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['Mas_Vnr_Type']==label, 1, 0)

Ames.drop("Mas_Vnr_Type", axis=1, inplace=True)
nominal.remove("Mas_Vnr_Type")

In [ ]:
# Foundation
top_3 = [x for x in Ames['Foundation'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['Foundation']==label, 1, 0)

Ames.drop("Foundation", axis=1, inplace=True)
nominal.remove("Foundation")

In [ ]:
# Heating
top_2 = [x for x in Ames['Heating'].value_counts().sort_values(ascending=False).head(2).index]
for label in top_2:
    Ames[label] = np.where(Ames['Heating']==label, 1, 0)

Ames.drop("Heating", axis=1, inplace=True)
nominal.remove("Heating")

In [ ]:
# Garage_Type
top_5 = [x for x in Ames['Garage_Type'].value_counts().sort_values(ascending=False).head(5).index]
for label in top_5:
    Ames[label] = np.where(Ames['Garage_Type']==label, 1, 0)

Ames.drop("Garage_Type", axis=1, inplace=True)
nominal.remove("Garage_Type")

In [ ]:
# Misc_Feature
top_2 = [x for x in Ames['Misc_Feature'].value_counts().sort_values(ascending=False).head(2).index]
for label in top_2:
    Ames[label] = np.where(Ames['Misc_Feature']==label, 1, 0)

Ames.drop("Misc_Feature", axis=1, inplace=True)
nominal.remove("Misc_Feature")

In [ ]:
# Sale_Type
top_3 = [x for x in Ames['Sale_Type'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['Sale_Type']==label, 1, 0)

Ames.drop("Sale_Type", axis=1, inplace=True)
nominal.remove("Sale_Type")

In [ ]:
# Sale_Condition
top_3 = [x for x in Ames['Sale_Condition'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    Ames[label] = np.where(Ames['Sale_Condition']==label, 1, 0)

Ames.drop("Sale_Condition", axis=1, inplace=True)
nominal.remove("Sale_Condition")

Ames_preprocessed = pd.get_dummies(Ames)

In [ ]:
def clip_outliers(variables):
    for var in variables:
        IQR = Ames[[var]].quantile(0.75) - Ames[[var]].quantile(0.25)
        Lower_fence = float(Ames[[var]].quantile(0.25) - (IQR * 3))
        Upper_fence = float(Ames[[var]].quantile(0.75) + (IQR * 3))
        Ames[var].clip(Lower_fence, Upper_fence, inplace=True)
    return Ames

Ames=clip_outliers(continous+discrete)

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

corr_features = correlation(Ames.drop(["Sale_Price"], axis=1), 0.7)
Ames.drop(labels=corr_features, axis=1, inplace=True)

In [ ]:
constant_features = [
    feat for feat in Ames.columns if Ames[feat].std() <= 0.1
]
Ames.drop(labels=constant_features, axis=1, inplace=True)
Ames=Ames.dropna()

scaler = RobustScaler().fit(Ames.drop(["Sale_Price"], axis=1))

In [ ]:
# Convert Sale_Price to log scale
Ames['Log_Sale_Price'] = np.log1p(Ames['Sale_Price'])

X = Ames.drop(['Sale_Price', 'Log_Sale_Price'], axis=1)
y = Ames['Log_Sale_Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=453111) 

In [ ]:
X_train.head()
# print(X_train['Alley'].unique())
# print(X_train['Street'].unique())


,PID,Lot_Frontage,Lot_Area,Lot_Shape,Land_Contour,Land_Slope,Overall_Qual,Overall_Cond,Year_Built,Year_Remod_Add,...,Norm,PosN,Gable,HdBoard,Wd Sdng,MetalSd,Plywood,BrkFace,BrkCmn,WD
582,534128010,0,10382.0,3,1,3,7,6,5,2,...,0,1,1,1,0,0,0,0,0,1
587,534129370,80,9600.0,4,1,3,5,5,4,2,...,0,0,1,1,0,0,0,1,0,1
1609,924100020,0,11075.0,3,1,2,5,4,4,2,...,1,0,1,1,0,0,0,1,0,1
260,907200290,0,10900.0,3,1,3,6,7,6,3,...,1,0,1,1,0,0,0,1,0,1
1920,535154080,75,9750.0,4,1,3,6,6,2,1,...,1,0,1,0,0,1,0,1,0,0


In [ ]:
y_train.head()

582     12.206078
587     12.001512
1609    12.055256
260     11.993813
1920    11.969724
Name: Log_Sale_Price, dtype: float64

In [ ]:
record_results =pd.DataFrame(index=range(0,10), columns=["lgb", "XGboost1", "XGboost2", "GradientBoosting"])
for step in range(0, 10):
    train_file="../data/train"+str(step)+".csv"
    test_file="../data/test"+str(step)+".csv"
    train_df= pd.read_csv(train_file, header=0, index_col="PID")
    train_df.drop("Garage_Yr_Blt", axis=1, inplace=True)
    train_df.dropna(inplace=True)
    trainY= train_df[["Sale_Price"]]
    trainX=train_df.drop(["Sale_Price","Condition_2","Utilities","Roof_Matl", "Latitude","Longitude"], axis=1)
    test_df=pd.read_csv(test_file, header=0, index_col="PID")
    test_df.drop("Garage_Yr_Blt", axis=1, inplace=True)
    test_df.dropna(inplace=True)
    testY= test_df[["Sale_Price"]]
    testX=test_df.drop(["Sale_Price","Condition_2","Utilities","Roof_Matl","Latitude","Longitude"], axis=1)
    Combine_df_X= pd.concat((trainX, testX))
    Combine_df_Y= pd.concat((trainY, testY))
    trainY=np.log1p(trainY)
    testY=np.log1p(testY)
    full_train= pd.concat([trainX, trainY], axis=1)
    full_test= pd.concat([testX, testY], axis=1)
    categorical = [var for var in Combine_df_X.columns if Combine_df_X[var].dtype=='O']
    numerical = [var for var in Combine_df_X.columns if Combine_df_X[var].dtype!='O']
    discrete = []
    continous=[]
    for var in numerical:
        if len(Combine_df_X[var].unique())<15:
            discrete.append(var)
        else:
            continous.append(var)
    discrete.append("Year_Built")
    discrete.append("Year_Remod_Add")
    discrete.remove("BsmtFin_SF_1")
    continous.remove("Year_Built")
    continous.remove("Year_Remod_Add")
    continous.append("BsmtFin_SF_1")
    
    ordinal=["Overall_Qual", "Overall_Cond", "Exter_Qual", "Exter_Cond","Bsmt_Qual","Bsmt_Cond", "Heating_QC","Kitchen_Qual","Fireplace_Qu","Garage_Qual","Garage_Cond",
         "Pool_QC","Lot_Shape","Land_Slope","Bsmt_Exposure","BsmtFin_Type_1","BsmtFin_Type_2","Electrical","Functional",
        "Paved_Drive","Fence"]
    nominal =[col for col in categorical if col not in ordinal]
    out=full_train.Gr_Liv_Area.sort_values(ascending=False)[:3].index
    full_train.drop(out, axis=0, inplace=True)
    out=full_train.Total_Bsmt_SF.sort_values(ascending=False)[:3].index
    full_train.drop(out, axis=0, inplace=True)
    
    
    full_train = full_train.replace({"Bsmt_Cond" : {"No_Basement" : 0, "Poor" : 1, "Fair" : 2, "Typical": 3, "Good" : 4, "Excellent" : 5},
                       "Bsmt_Exposure" : {"No_Basement":0,"No" : 1, "Mn" : 2, "Av": 3, "Gd" : 4},
                       "BsmtFin_Type_1" : {"No_Basement" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtFin_Type_2" : {"No_Basement" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "Bsmt_Qual" : {"No_Basement" : 0, "Poor" : 1, "Fair" : 2, "Typical": 3, "Good" : 4, "Excellent" : 5},
                       "Exter_Cond" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                       "Exter_Qual" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                       "Fireplace_Qu" : {"No_Fireplace" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                       "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8},
                                                "Garage_Cond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "Garage_Qual" : {"No_Garage" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                       "Heating_QC" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                       "Kitchen_Qual" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                       "Land_Slope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                       "Lot_Shape" : {"Irregular" : 1, "Moderately_Irregular" : 2, "Slightly_Irregular" : 3, "Regular" : 4},
                       "Paved_Drive" : {"Dirt_Gravel" : 0, "Paved" : 1, "Partial_Pavement" : 2},
                       "Pool_QC" : {"No_Pool" : 0, "Fair" : 1, "Typical" : 2, "Good" : 3, "Excellent" : 4},
                      "Overall_Qual":{"Very_Excellent": 10, "Excellent":9,"Very_Good":8, "Good":7, "Above_Average":6, "Average": 5, "Below_Average":4, "Fair": 3,"Poor":2, "Very_Poor":1},
                      "Overall_Cond" :{"Very_Excellent": 10, "Excellent":9,"Very_Good":8, "Good":7, "Above_Average":6, "Average": 5, "Below_Average":4, "Fair": 3,"Poor":2, "Very_Poor":1},
                      "Electrical ": {"Unknown": 0,"SBrkr ": 1,"FuseA": 2, "FuseF": 3, "FuseP": 4,"Mix": 5 },
                       "Fence":{"No_Fence":0, "Minimum_Wood_Wire":1,"Good_Wood":2,"Minimum_Privacy":3,"Good_Privacy":4},
                        "Garage_Cond" : {"No_Garage" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                      })

    full_train["Electrical"].replace({"Unknown": 0,"SBrkr": 1,"FuseA": 2, "FuseF": 3, "FuseP": 4,"Mix": 5 }, inplace=True)

    full_test = full_test.replace({"Bsmt_Cond" : {"No_Basement" : 0, "Poor" : 1, "Fair" : 2, "Typical": 3, "Good" : 4, "Excellent" : 5},
                           "Bsmt_Exposure" : {"No_Basement":0,"No" : 1, "Mn" : 2, "Av": 3, "Gd" : 4},
                           "BsmtFin_Type_1" : {"No_Basement" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                             "ALQ" : 5, "GLQ" : 6},
                           "BsmtFin_Type_2" : {"No_Basement" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                             "ALQ" : 5, "GLQ" : 6},
                           "Bsmt_Qual" : {"No_Basement" : 0, "Poor" : 1, "Fair" : 2, "Typical": 3, "Good" : 4, "Excellent" : 5},
                           "Exter_Cond" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                           "Exter_Qual" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                           "Fireplace_Qu" : {"No_Fireplace" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                           "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                           "Min2" : 6, "Min1" : 7, "Typ" : 8},
                                                    "Garage_Cond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                           "Garage_Qual" : {"No_Garage" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                           "Heating_QC" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                           "Kitchen_Qual" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                           "Land_Slope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                           "Lot_Shape" : {"Irregular" : 1, "Moderately_Irregular" : 2, "Slightly_Irregular" : 3, "Regular" : 4},
                           "Paved_Drive" : {"Dirt_Gravel" : 0, "Paved" : 1, "Partial_Pavement" : 2},
                           "Pool_QC" : {"No_Pool" : 0, "Fair" : 1, "Typical" : 2, "Good" : 3, "Excellent" : 4},
                          "Overall_Qual":{"Very_Excellent": 10, "Excellent":9,"Very_Good":8, "Good":7, "Above_Average":6, "Average": 5, "Below_Average":4, "Fair": 3,"Poor":2, "Very_Poor":1},
                          "Overall_Cond" :{"Very_Excellent": 10, "Excellent":9,"Very_Good":8, "Good":7, "Above_Average":6, "Average": 5, "Below_Average":4, "Fair": 3,"Poor":2, "Very_Poor":1},
                          "Electrical ": {"Unknown": 0,"SBrkr ": 1,"FuseA": 2, "FuseF": 3, "FuseP": 4,"Mix": 5 },
                           "Fence":{"No_Fence":0, "Minimum_Wood_Wire":1,"Good_Wood":2,"Minimum_Privacy":3,"Good_Privacy":4},
                            "Garage_Cond" : {"No_Garage" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                          })
    full_test["Electrical"].replace({"Unknown": 0,"SBrkr": 1,"FuseA": 2, "FuseF": 3, "FuseP": 4,"Mix": 5 }, inplace=True)
    
    
    Age_range = trainX['Year_Built'].max() - trainX['Year_Built'].min()
    min_value = int(np.floor(trainX['Year_Built'].min()))
    max_value = int(np.ceil(trainX['Year_Built'].max()))

    # let's round the bin width
    inter_value = int(np.round(Age_range/9))

    intervals = [i for i in range(min_value, max_value+inter_value, inter_value)]
    labels=[]
    for i in range(1, len(intervals)):
        labels.append(i)

    full_train['Year_Built'] = pd.cut(x = full_train['Year_Built'], bins=intervals, labels=labels, include_lowest=True)
    full_test['Year_Built'] = pd.cut(x = full_test['Year_Built'], bins=intervals, labels=labels, include_lowest=True)
    full_train['Year_Built']=full_train['Year_Built'].astype('int64', copy=False,errors="ignore")
    full_test['Year_Built']=full_test['Year_Built'].astype('int64', copy=False,errors="ignore")

    Age_range = trainX['Year_Remod_Add'].max() - trainX['Year_Remod_Add'].min()
    min_value = int(np.floor(trainX['Year_Remod_Add'].min()))
    max_value = int(np.ceil(trainX['Year_Remod_Add'].max()))

    # let's round the bin width
    inter_value = int(np.round(Age_range/5))

    intervals = [i for i in range(min_value, max_value+inter_value, inter_value)]
    labels=[]
    for i in range(1, len(intervals)):
        labels.append(i)

    full_train['Year_Remod_Add'] = pd.cut(x = full_train['Year_Remod_Add'], bins=intervals, labels=labels, include_lowest=True)
    full_test['Year_Remod_Add'] = pd.cut(x = full_test['Year_Remod_Add'], bins=intervals, labels=labels, include_lowest=True)
    full_train['Year_Remod_Add']=full_train['Year_Remod_Add'].astype('int64', copy=False,errors="ignore")
    full_test['Year_Remod_Add']=full_test['Year_Remod_Add'].astype('int64', copy=False,errors="ignore")
    
    
    train_objs_num = len(full_train)
    dataset = pd.concat(objs=[full_train, full_test], axis=0)

    #MS_Subclass
    top_5 = [x for x in dataset['MS_SubClass'].value_counts().sort_values(ascending=False).head(5).index]
    for label in top_5:
        dataset[label] = np.where(dataset['MS_SubClass']==label, 1, 0)

    dataset.drop("MS_SubClass", axis=1, inplace=True)

    nominal.remove("MS_SubClass")

    #MS_Zoning
    top_3 = [x for x in dataset['MS_Zoning'].value_counts().sort_values(ascending=False).head(3).index]
    for label in top_3:
        dataset[label] = np.where(dataset['MS_Zoning']==label, 1, 0)

    dataset.drop("MS_Zoning", axis=1, inplace=True)

    nominal.remove("MS_Zoning")

    #Lot_Config
    top_3 = [x for x in dataset['Lot_Config'].value_counts().sort_values(ascending=False).head(3).index]
    for label in top_3:
        dataset[label] = np.where(dataset['Lot_Config']==label, 1, 0)

    dataset.drop("Lot_Config", axis=1, inplace=True)

    nominal.remove("Lot_Config")

    #Neighborhood 
    top_8 = [x for x in dataset['Neighborhood'].value_counts().sort_values(ascending=False).head(8).index]
    for label in top_8:
        dataset[label] = np.where(dataset['Neighborhood']==label, 1, 0)

    dataset.drop("Neighborhood", axis=1, inplace=True)
    nominal.remove("Neighborhood")

    #Condition_1
    top_3 = [x for x in dataset['Condition_1'].value_counts().sort_values(ascending=False).head(3).index]
    for label in top_3:
        dataset[label] = np.where(dataset['Condition_1']==label, 1, 0)

    dataset.drop("Condition_1", axis=1, inplace=True)
    nominal.remove("Condition_1")

    #House_Style 
    top_3 = [x for x in dataset['House_Style'].value_counts().sort_values(ascending=False).head(3).index]
    for label in top_3:
        dataset[label] = np.where(dataset['House_Style']==label, 1, 0)

    dataset.drop("House_Style", axis=1, inplace=True)
    nominal.remove("House_Style")

    #Roof_Style
    top_2 = [x for x in dataset['Roof_Style'].value_counts().sort_values(ascending=False).head(2).index]
    for label in top_2:
        dataset[label] = np.where(dataset['Roof_Style']==label, 1, 0)

    dataset.drop("Roof_Style", axis=1, inplace=True)
    nominal.remove("Roof_Style")

    #Exterior_1st 
    top_5 = [x for x in dataset['Exterior_1st'].value_counts().sort_values(ascending=False).head(5).index]
    for label in top_5:
        dataset[label] = np.where(dataset['Exterior_1st']==label, 1, 0)

    dataset.drop("Exterior_1st", axis=1, inplace=True)
    nominal.remove("Exterior_1st")

    #Exterior_2nd 
    top_5 = [x for x in dataset['Exterior_2nd'].value_counts().sort_values(ascending=False).head(5).index]
    for label in top_5:
        dataset[label] = np.where(dataset['Exterior_2nd']==label, 1, 0)

    dataset.drop("Exterior_2nd", axis=1, inplace=True)
    nominal.remove("Exterior_2nd")

    #Mas_Vnr_Type
    top_3 = [x for x in dataset['Mas_Vnr_Type'].value_counts().sort_values(ascending=False).head(3).index]
    for label in top_3:
        dataset[label] = np.where(dataset['Mas_Vnr_Type']==label, 1, 0)

    dataset.drop("Mas_Vnr_Type", axis=1, inplace=True)
    nominal.remove("Mas_Vnr_Type")

    #Foundation
    top_3 = [x for x in dataset['Foundation'].value_counts().sort_values(ascending=False).head(3).index]
    for label in top_3:
        dataset[label] = np.where(dataset['Foundation']==label, 1, 0)

    dataset.drop("Foundation", axis=1, inplace=True)
    nominal.remove("Foundation")

    #Heating
    top_2 = [x for x in dataset['Heating'].value_counts().sort_values(ascending=False).head(2).index]
    for label in top_2:
        dataset[label] = np.where(dataset['Heating']==label, 1, 0)

    dataset.drop("Heating", axis=1, inplace=True)
    nominal.remove("Heating")

    #Garage_Type
    top_5 = [x for x in dataset['Garage_Type'].value_counts().sort_values(ascending=False).head(5).index]
    for label in top_5:
        dataset[label] = np.where(dataset['Garage_Type']==label, 1, 0)

    dataset.drop("Garage_Type", axis=1, inplace=True)
    nominal.remove("Garage_Type")

    #Misc_Feature
    top_2 = [x for x in dataset['Misc_Feature'].value_counts().sort_values(ascending=False).head(2).index]
    for label in top_2:
        dataset[label] = np.where(dataset['Misc_Feature']==label, 1, 0)

    dataset.drop("Misc_Feature", axis=1, inplace=True)
    nominal.remove("Misc_Feature")

    #Sale_Type
    top_3 = [x for x in dataset['Sale_Type'].value_counts().sort_values(ascending=False).head(3).index]
    for label in top_3:
        dataset[label] = np.where(dataset['Sale_Type']==label, 1, 0)

    dataset.drop("Sale_Type", axis=1, inplace=True)
    nominal.remove("Sale_Type")

    #Sale_Condition
    top_3 = [x for x in dataset['Sale_Condition'].value_counts().sort_values(ascending=False).head(3).index]
    for label in top_3:
        dataset[label] = np.where(dataset['Sale_Condition']==label, 1, 0)

    dataset.drop("Sale_Condition", axis=1, inplace=True)
    nominal.remove("Sale_Condition")
    
    dataset_preprocessed = pd.get_dummies(dataset)
    full_train = dataset_preprocessed[:train_objs_num]
    full_test = dataset_preprocessed[train_objs_num:]
    
    def clip_outliers(variables):
        for var in variables:
            IQR = full_train[[var]].quantile(0.75) - full_train[[var]].quantile(0.25)
            Lower_fence = float(full_train[[var]].quantile(0.25) - (IQR * 3))
            Upper_fence = float(full_train[[var]].quantile(0.75) + (IQR * 3))
            full_train[var].clip(Lower_fence, Upper_fence, inplace=True)
        return full_train

    full_train=clip_outliers(continous+discrete)

    def correlation(dataset, threshold):
        col_corr = set()  # Set of all the names of correlated columns
        corr_matrix = dataset.corr()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                    colname = corr_matrix.columns[i]  # getting the name of column
                    col_corr.add(colname)
        return col_corr

    corr_features = correlation(full_train.drop(["Sale_Price"], axis=1), 0.7)
    full_train.drop(labels=corr_features, axis=1, inplace=True)
    full_test.drop(labels=corr_features, axis=1, inplace=True)

    constant_features = [
        feat for feat in full_train.columns if full_train[feat].std() <= 0.1
    ]
    full_train.drop(labels=constant_features, axis=1, inplace=True)
    full_test.drop(labels=constant_features, axis=1, inplace=True)
    full_test=full_test.dropna()
    full_train=full_train.dropna()

    trainY= full_train[["Sale_Price"]]
    trainX=full_train.drop(["Sale_Price"], axis=1)
    testY= full_test[["Sale_Price"]]
    testX=full_test.drop(["Sale_Price"], axis=1)

    scaler = RobustScaler().fit(trainX)
    scaled_train_X=scaler.transform(trainX)
    scaled_test_X=scaler.transform(testX)
    
    
    xgb_reg_model = xgb.XGBRegressor(
                     colsample_bytree=0.2,
                     gamma=0.0,
                     learning_rate=0.01,
                     max_depth=4,
                     min_child_weight=1.5,
                     n_estimators=8000,                                                                  
                     reg_alpha=0.9,
                     reg_lambda=0.6,
                     subsample=0.2,
                     silent=1)
    xgb_reg_model.fit(scaled_train_X, trainY)
    y_pred_train = xgb_reg_model.predict(scaled_train_X)
    y_pred_test = xgb_reg_model.predict(scaled_test_X)                                    
    record_results.loc[step, "XGboost1"]=np.sqrt(mean_squared_error(y_pred_test, testY))
    
    
    
    


    
    Gra_boo_model = ensemble.GradientBoostingRegressor( learning_rate= 0.01, max_depth= 4, min_samples_leaf=12, min_samples_split= 5, n_estimators= 3000,max_features='sqrt', loss='huber')
    Gra_boo_model.fit(scaled_train_X, trainY)
    y_pred_test = Gra_boo_model.predict(scaled_test_X)                                    
    record_results.loc[step, "GradientBoosting"]=np.sqrt(mean_squared_error(y_pred_test, testY))
    
    print("Iteration "+str(step) +" ended")
record_results    

Iteration 0 ended
Iteration 1 ended
Iteration 2 ended
Iteration 3 ended


KeyboardInterrupt: 

In [ ]:
record_results.mean()

lgb                      NaN
XGboost1            0.118888
XGboost2            0.124518
GradientBoosting    0.121640
dtype: float64

In [ ]:
train_df= pd.read_csv("../data/train3.csv", header=0, index_col="PID")
train_df.drop("Garage_Yr_Blt", axis=1, inplace=True)
train_df.dropna(inplace=True)
trainY= train_df[["Sale_Price"]]
trainX=train_df.drop(["Sale_Price","Condition_2","Utilities","Roof_Matl", "Latitude","Longitude"], axis=1)



test_df=pd.read_csv("../data/test3.csv", header=0, index_col="PID")
test_df.drop("Garage_Yr_Blt", axis=1, inplace=True)
test_df.dropna(inplace=True)
testY= test_df[["Sale_Price"]]
testX=test_df.drop(["Sale_Price","Condition_2","Utilities","Roof_Matl","Latitude","Longitude"], axis=1)

Combine_df_X= pd.concat((trainX, testX))
Combine_df_Y= pd.concat((trainY, testY))

trainY=np.log1p(trainY)
testY=np.log1p(testY)


full_train= pd.concat([trainX, trainY], axis=1)
full_test= pd.concat([testX, testY], axis=1)
    
    

In [ ]:
categorical = [var for var in Combine_df_X.columns if Combine_df_X[var].dtype=='O']
numerical = [var for var in Combine_df_X.columns if Combine_df_X[var].dtype!='O']
 
discrete = []
continous=[]
for var in numerical:
    if len(Combine_df_X[var].unique())<15:
        discrete.append(var)
    else:
        continous.append(var)

discrete.append("Year_Built")
discrete.append("Year_Remod_Add")
discrete.remove("BsmtFin_SF_1")

continous.remove("Year_Built")
continous.remove("Year_Remod_Add")
continous.append("BsmtFin_SF_1")


In [ ]:
ordinal=["Overall_Qual", "Overall_Cond", "Exter_Qual", "Exter_Cond","Bsmt_Qual","Bsmt_Cond", "Heating_QC","Kitchen_Qual","Fireplace_Qu","Garage_Qual","Garage_Cond",
         "Pool_QC","Lot_Shape","Land_Slope","Bsmt_Exposure","BsmtFin_Type_1","BsmtFin_Type_2","Electrical","Functional",
        "Paved_Drive","Fence"]
nominal =[col for col in categorical if col not in ordinal]



out=full_train.Gr_Liv_Area.sort_values(ascending=False)[:3].index
full_train.drop(out, axis=0, inplace=True)


out=full_train.Total_Bsmt_SF.sort_values(ascending=False)[:3].index
full_train.drop(out, axis=0, inplace=True)

In [ ]:
full_train = full_train.replace({"Bsmt_Cond" : {"No_Basement" : 0, "Poor" : 1, "Fair" : 2, "Typical": 3, "Good" : 4, "Excellent" : 5},
                       "Bsmt_Exposure" : {"No_Basement":0,"No" : 1, "Mn" : 2, "Av": 3, "Gd" : 4},
                       "BsmtFin_Type_1" : {"No_Basement" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtFin_Type_2" : {"No_Basement" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "Bsmt_Qual" : {"No_Basement" : 0, "Poor" : 1, "Fair" : 2, "Typical": 3, "Good" : 4, "Excellent" : 5},
                       "Exter_Cond" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                       "Exter_Qual" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                       "Fireplace_Qu" : {"No_Fireplace" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                       "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8},
                                                "Garage_Cond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "Garage_Qual" : {"No_Garage" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                       "Heating_QC" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                       "Kitchen_Qual" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                       "Land_Slope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                       "Lot_Shape" : {"Irregular" : 1, "Moderately_Irregular" : 2, "Slightly_Irregular" : 3, "Regular" : 4},
                       "Paved_Drive" : {"Dirt_Gravel" : 0, "Paved" : 1, "Partial_Pavement" : 2},
                       "Pool_QC" : {"No_Pool" : 0, "Fair" : 1, "Typical" : 2, "Good" : 3, "Excellent" : 4},
                      "Overall_Qual":{"Very_Excellent": 10, "Excellent":9,"Very_Good":8, "Good":7, "Above_Average":6, "Average": 5, "Below_Average":4, "Fair": 3,"Poor":2, "Very_Poor":1},
                      "Overall_Cond" :{"Very_Excellent": 10, "Excellent":9,"Very_Good":8, "Good":7, "Above_Average":6, "Average": 5, "Below_Average":4, "Fair": 3,"Poor":2, "Very_Poor":1},
                      "Electrical ": {"Unknown": 0,"SBrkr ": 1,"FuseA": 2, "FuseF": 3, "FuseP": 4,"Mix": 5 },
                       "Fence":{"No_Fence":0, "Minimum_Wood_Wire":1,"Good_Wood":2,"Minimum_Privacy":3,"Good_Privacy":4},
                        "Garage_Cond" : {"No_Garage" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                      })

full_train["Electrical"].replace({"Unknown": 0,"SBrkr": 1,"FuseA": 2, "FuseF": 3, "FuseP": 4,"Mix": 5 }, inplace=True)

full_test = full_test.replace({"Bsmt_Cond" : {"No_Basement" : 0, "Poor" : 1, "Fair" : 2, "Typical": 3, "Good" : 4, "Excellent" : 5},
                       "Bsmt_Exposure" : {"No_Basement":0,"No" : 1, "Mn" : 2, "Av": 3, "Gd" : 4},
                       "BsmtFin_Type_1" : {"No_Basement" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtFin_Type_2" : {"No_Basement" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "Bsmt_Qual" : {"No_Basement" : 0, "Poor" : 1, "Fair" : 2, "Typical": 3, "Good" : 4, "Excellent" : 5},
                       "Exter_Cond" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                       "Exter_Qual" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                       "Fireplace_Qu" : {"No_Fireplace" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                       "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8},
                                                "Garage_Cond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "Garage_Qual" : {"No_Garage" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                       "Heating_QC" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                       "Kitchen_Qual" : {"Poor" : 1, "Fair" : 2, "Typical": 3, "Good": 4, "Excellent" : 5},
                       "Land_Slope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                       "Lot_Shape" : {"Irregular" : 1, "Moderately_Irregular" : 2, "Slightly_Irregular" : 3, "Regular" : 4},
                       "Paved_Drive" : {"Dirt_Gravel" : 0, "Paved" : 1, "Partial_Pavement" : 2},
                       "Pool_QC" : {"No_Pool" : 0, "Fair" : 1, "Typical" : 2, "Good" : 3, "Excellent" : 4},
                      "Overall_Qual":{"Very_Excellent": 10, "Excellent":9,"Very_Good":8, "Good":7, "Above_Average":6, "Average": 5, "Below_Average":4, "Fair": 3,"Poor":2, "Very_Poor":1},
                      "Overall_Cond" :{"Very_Excellent": 10, "Excellent":9,"Very_Good":8, "Good":7, "Above_Average":6, "Average": 5, "Below_Average":4, "Fair": 3,"Poor":2, "Very_Poor":1},
                      "Electrical ": {"Unknown": 0,"SBrkr ": 1,"FuseA": 2, "FuseF": 3, "FuseP": 4,"Mix": 5 },
                       "Fence":{"No_Fence":0, "Minimum_Wood_Wire":1,"Good_Wood":2,"Minimum_Privacy":3,"Good_Privacy":4},
                        "Garage_Cond" : {"No_Garage" : 0, "Poor" : 1, "Fair" : 2, "Typical" : 3, "Good" : 4, "Excellent" : 5},
                      })
full_test["Electrical"].replace({"Unknown": 0,"SBrkr": 1,"FuseA": 2, "FuseF": 3, "FuseP": 4,"Mix": 5 }, inplace=True)

In [ ]:
Age_range = trainX['Year_Built'].max() - trainX['Year_Built'].min()
min_value = int(np.floor(trainX['Year_Built'].min()))
max_value = int(np.ceil(trainX['Year_Built'].max()))
 
# let's round the bin width
inter_value = int(np.round(Age_range/9))

intervals = [i for i in range(min_value, max_value+inter_value, inter_value)]
labels=[]
for i in range(1, len(intervals)):
    labels.append(i)

full_train['Year_Built'] = pd.cut(x = full_train['Year_Built'], bins=intervals, labels=labels, include_lowest=True)
full_test['Year_Built'] = pd.cut(x = full_test['Year_Built'], bins=intervals, labels=labels, include_lowest=True)
full_train['Year_Built']=full_train['Year_Built'].astype('int64', copy=False,errors="ignore")
full_test['Year_Built']=full_test['Year_Built'].astype('int64', copy=False,errors="ignore")




Age_range = trainX['Year_Remod_Add'].max() - trainX['Year_Remod_Add'].min()
min_value = int(np.floor(trainX['Year_Remod_Add'].min()))
max_value = int(np.ceil(trainX['Year_Remod_Add'].max()))
 
# let's round the bin width
inter_value = int(np.round(Age_range/5))

intervals = [i for i in range(min_value, max_value+inter_value, inter_value)]
labels=[]
for i in range(1, len(intervals)):
    labels.append(i)

full_train['Year_Remod_Add'] = pd.cut(x = full_train['Year_Remod_Add'], bins=intervals, labels=labels, include_lowest=True)
full_test['Year_Remod_Add'] = pd.cut(x = full_test['Year_Remod_Add'], bins=intervals, labels=labels, include_lowest=True)
full_train['Year_Remod_Add']=full_train['Year_Remod_Add'].astype('int64', copy=False,errors="ignore")
full_test['Year_Remod_Add']=full_test['Year_Remod_Add'].astype('int64', copy=False,errors="ignore")

In [ ]:
high_card=[]
for col in nominal:
    if len(Combine_df_X[col].unique()) >7:
        high_card.append(col)

In [ ]:
train_objs_num = len(full_train)
dataset = pd.concat(objs=[full_train, full_test], axis=0)

#MS_Subclass
top_5 = [x for x in dataset['MS_SubClass'].value_counts().sort_values(ascending=False).head(5).index]
for label in top_5:
    dataset[label] = np.where(dataset['MS_SubClass']==label, 1, 0)

dataset.drop("MS_SubClass", axis=1, inplace=True)

nominal.remove("MS_SubClass")

#MS_Zoning
top_3 = [x for x in dataset['MS_Zoning'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    dataset[label] = np.where(dataset['MS_Zoning']==label, 1, 0)

dataset.drop("MS_Zoning", axis=1, inplace=True)

nominal.remove("MS_Zoning")

#Lot_Config
top_3 = [x for x in dataset['Lot_Config'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    dataset[label] = np.where(dataset['Lot_Config']==label, 1, 0)

dataset.drop("Lot_Config", axis=1, inplace=True)

nominal.remove("Lot_Config")

#Neighborhood 
top_8 = [x for x in dataset['Neighborhood'].value_counts().sort_values(ascending=False).head(8).index]
for label in top_8:
    dataset[label] = np.where(dataset['Neighborhood']==label, 1, 0)

dataset.drop("Neighborhood", axis=1, inplace=True)
nominal.remove("Neighborhood")

#Condition_1
top_3 = [x for x in dataset['Condition_1'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    dataset[label] = np.where(dataset['Condition_1']==label, 1, 0)

dataset.drop("Condition_1", axis=1, inplace=True)
nominal.remove("Condition_1")

#House_Style 
top_3 = [x for x in dataset['House_Style'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    dataset[label] = np.where(dataset['House_Style']==label, 1, 0)

dataset.drop("House_Style", axis=1, inplace=True)
nominal.remove("House_Style")

#Roof_Style
top_2 = [x for x in dataset['Roof_Style'].value_counts().sort_values(ascending=False).head(2).index]
for label in top_2:
    dataset[label] = np.where(dataset['Roof_Style']==label, 1, 0)

dataset.drop("Roof_Style", axis=1, inplace=True)
nominal.remove("Roof_Style")

#Exterior_1st 
top_5 = [x for x in dataset['Exterior_1st'].value_counts().sort_values(ascending=False).head(5).index]
for label in top_5:
    dataset[label] = np.where(dataset['Exterior_1st']==label, 1, 0)

dataset.drop("Exterior_1st", axis=1, inplace=True)
nominal.remove("Exterior_1st")

#Exterior_2nd 
top_5 = [x for x in dataset['Exterior_2nd'].value_counts().sort_values(ascending=False).head(5).index]
for label in top_5:
    dataset[label] = np.where(dataset['Exterior_2nd']==label, 1, 0)

dataset.drop("Exterior_2nd", axis=1, inplace=True)
nominal.remove("Exterior_2nd")

#Mas_Vnr_Type
top_3 = [x for x in dataset['Mas_Vnr_Type'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    dataset[label] = np.where(dataset['Mas_Vnr_Type']==label, 1, 0)

dataset.drop("Mas_Vnr_Type", axis=1, inplace=True)
nominal.remove("Mas_Vnr_Type")

#Foundation
top_3 = [x for x in dataset['Foundation'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    dataset[label] = np.where(dataset['Foundation']==label, 1, 0)

dataset.drop("Foundation", axis=1, inplace=True)
nominal.remove("Foundation")

#Heating
top_2 = [x for x in dataset['Heating'].value_counts().sort_values(ascending=False).head(2).index]
for label in top_2:
    dataset[label] = np.where(dataset['Heating']==label, 1, 0)

dataset.drop("Heating", axis=1, inplace=True)
nominal.remove("Heating")

#Garage_Type
top_5 = [x for x in dataset['Garage_Type'].value_counts().sort_values(ascending=False).head(5).index]
for label in top_5:
    dataset[label] = np.where(dataset['Garage_Type']==label, 1, 0)

dataset.drop("Garage_Type", axis=1, inplace=True)
nominal.remove("Garage_Type")

#Misc_Feature
top_2 = [x for x in dataset['Misc_Feature'].value_counts().sort_values(ascending=False).head(2).index]
for label in top_2:
    dataset[label] = np.where(dataset['Misc_Feature']==label, 1, 0)

dataset.drop("Misc_Feature", axis=1, inplace=True)
nominal.remove("Misc_Feature")

#Sale_Type
top_3 = [x for x in dataset['Sale_Type'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    dataset[label] = np.where(dataset['Sale_Type']==label, 1, 0)

dataset.drop("Sale_Type", axis=1, inplace=True)
nominal.remove("Sale_Type")

#Sale_Condition
top_3 = [x for x in dataset['Sale_Condition'].value_counts().sort_values(ascending=False).head(3).index]
for label in top_3:
    dataset[label] = np.where(dataset['Sale_Condition']==label, 1, 0)

dataset.drop("Sale_Condition", axis=1, inplace=True)
nominal.remove("Sale_Condition")

In [ ]:
dataset_preprocessed = pd.get_dummies(dataset)
full_train = dataset_preprocessed[:train_objs_num]
full_test = dataset_preprocessed[train_objs_num:]

In [ ]:
def clip_outliers(variables):
    for var in variables:
        IQR = full_train[[var]].quantile(0.75) - full_train[[var]].quantile(0.25)
        Lower_fence = float(full_train[[var]].quantile(0.25) - (IQR * 3))
        Upper_fence = float(full_train[[var]].quantile(0.75) + (IQR * 3))
        full_train[var].clip(Lower_fence, Upper_fence, inplace=True)
    return full_train

full_train=clip_outliers(continous+discrete)


def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr


corr_features = correlation(full_train.drop(["Sale_Price"], axis=1), 0.7)
full_train.drop(labels=corr_features, axis=1, inplace=True)
full_test.drop(labels=corr_features, axis=1, inplace=True)

constant_features = [
    feat for feat in full_train.columns if full_train[feat].std() <= 0.1
]

full_train.drop(labels=constant_features, axis=1, inplace=True)
full_test.drop(labels=constant_features, axis=1, inplace=True)

full_test=full_test.dropna()
full_train=full_train.dropna()

trainY= full_train[["Sale_Price"]]
trainX=full_train.drop(["Sale_Price"], axis=1)

testY= full_test[["Sale_Price"]]
testX=full_test.drop(["Sale_Price"], axis=1)

scaler = preprocessing.StandardScaler().fit(trainX)

scaled_train_X=scaler.transform(trainX)
scaled_test_X=scaler.transform(testX)

In [ ]:
print(scaled_test_X.shape)

(879, 88)


In [ ]:
def rmsle_cv(model, train_X, train_Y):
    kf = KFold(5, shuffle=True, random_state=3064).get_n_splits(train_X)
    rmse= np.sqrt(-cross_val_score(model, train_X, train_Y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
regr = Lasso(alpha= 0.0009, max_iter=50000)
regr.fit(scaled_train_X, trainY)

pred_train= regr.predict(scaled_train_X)
pred_test = regr.predict(scaled_test_X)

#score=rmsle_cv(regr, scaled_train_X, trainY)
#print("Cross validated error on train set ",score.mean())

#score=rmsle_cv(regr, scaled_test_X, testY)
#print("Cross validated error on test set ",score.mean())

print('R2 on train data: {}'.format(r2_score(pred_train, trainY)))
print('RMSE on train data: {}'.format(np.sqrt(mean_squared_error(pred_train, trainY))))
print('R2 on test data: {}'.format(r2_score(pred_test, testY)))
print('RMSE on test data: {}'.format(np.sqrt(mean_squared_error(pred_test, testY))))

R2 on train data: 0.9182831005770815
RMSE on train data: 0.10989354379675151
R2 on test data: 0.898415488969318
RMSE on test data: 0.12784073180777877


In [ ]:
param_grid={'n_estimators':[5000, 4000, 3000], 'learning_rate':[0.05, 0.04, 0.03, 0.02], 'max_depth':[3,4,5,6,7],
           "min_samples_leaf":[10,12,15,20,25], 'min_samples_split':[5,10,15]}
Gra_boo_model = ensemble.GradientBoostingRegressor( max_features='sqrt', loss='huber')

clf = GridSearchCV(Gra_boo_model, param_grid, cv=5,verbose=2,n_jobs=8)
clf.fit(scaled_train_X, trainY)
print(clf.best_params_)
pred_train=clf.predict(scaled_train_X)
pred_test= clf.predict(scaled_test_X)

#score=rmsle_cv(Gra_boo_model, scaled_train_X, trainY)
#print("Cross validated error on train set ",score.mean())

#score=rmsle_cv(Gra_boo_model, scaled_test_X, testY)
#print("Cross validated error on test set ",score.mean())

print('R2 on train data: {}'.format(r2_score(pred_train, trainY)))
print('RMSE on train data: {}'.format(np.sqrt(mean_squared_error(pred_train, trainY))))
print('R2 on test data: {}'.format(r2_score(pred_test, testY)))
print('RMSE on test data: {}'.format(np.sqrt(mean_squared_error(pred_test, testY))))

Fitting 5 folds for each of 900 candidates, totalling 4500 fits
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=5, n_estimators=5000 
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=5, n_estimators=5000 
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=5, n_estimators=5000 
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=5, n_estimators=5000 
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=5, n_estimators=4000 
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=5, n_estimators=5000 
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=5, n_estimators=4000 
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=5, n_estimators=4000 
[CV]  learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=5, n_estimators=4000, total=  35.5s
[CV]  learning_rate=0.05, max_

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  2.4min


[CV]  learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=10, n_estimators=3000, total=  28.7s
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=15, n_estimators=5000 
[CV]  learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=10, n_estimators=3000, total=  28.4s
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=15, n_estimators=5000 
[CV]  learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=10, n_estimators=3000, total=  27.7s
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=15, n_estimators=4000 
[CV]  learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=10, n_estimators=3000, total=  27.8s
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=15, n_estimators=4000 
[CV]  learning_rate=0.05, max_depth=3, min_samples_leaf=10, min_samples_split=10, n_estimators=4000, total=  37.4s
[CV] learning_rate=0.05, 

[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=12, min_samples_split=10, n_estimators=3000 
[CV]  learning_rate=0.05, max_depth=3, min_samples_leaf=12, min_samples_split=10, n_estimators=4000, total=  38.1s
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=12, min_samples_split=10, n_estimators=3000 
[CV]  learning_rate=0.05, max_depth=3, min_samples_leaf=12, min_samples_split=10, n_estimators=5000, total=  46.2s
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=12, min_samples_split=10, n_estimators=3000 
[CV]  learning_rate=0.05, max_depth=3, min_samples_leaf=12, min_samples_split=10, n_estimators=5000, total=  45.9s
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=12, min_samples_split=10, n_estimators=3000 
[CV]  learning_rate=0.05, max_depth=3, min_samples_leaf=12, min_samples_split=10, n_estimators=5000, total=  45.8s
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=12, min_samples_split=15, n_estimators=5000 
[CV]  learning_rate=0.05, max_depth=3, m

[CV]  learning_rate=0.05, max_depth=3, min_samples_leaf=15, min_samples_split=5, n_estimators=3000, total=  26.8s
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=15, min_samples_split=10, n_estimators=4000 
[CV]  learning_rate=0.05, max_depth=3, min_samples_leaf=15, min_samples_split=5, n_estimators=4000, total=  36.2s
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=15, min_samples_split=10, n_estimators=4000 
[CV]  learning_rate=0.05, max_depth=3, min_samples_leaf=15, min_samples_split=5, n_estimators=3000, total=  26.8s
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=15, min_samples_split=10, n_estimators=4000 
[CV]  learning_rate=0.05, max_depth=3, min_samples_leaf=15, min_samples_split=10, n_estimators=5000, total=  42.8s
[CV] learning_rate=0.05, max_depth=3, min_samples_leaf=15, min_samples_split=10, n_estimators=4000 
[CV]  learning_rate=0.05, max_depth=3, min_samples_leaf=15, min_samples_split=10, n_estimators=5000, total=  41.1s
[CV] learning_rate=0.05, max

...

[CV]  learning_rate=0.02, max_depth=7, min_samples_leaf=20, min_samples_split=5, n_estimators=3000, total=  56.4s
[CV] learning_rate=0.02, max_depth=7, min_samples_leaf=20, min_samples_split=10, n_estimators=5000 
[CV]  learning_rate=0.02, max_depth=7, min_samples_leaf=20, min_samples_split=5, n_estimators=3000, total=  56.7s
[CV] learning_rate=0.02, max_depth=7, min_samples_leaf=20, min_samples_split=10, n_estimators=5000 
[CV]  learning_rate=0.02, max_depth=7, min_samples_leaf=20, min_samples_split=5, n_estimators=3000, total=  55.9s
[CV] learning_rate=0.02, max_depth=7, min_samples_leaf=20, min_samples_split=10, n_estimators=5000 
[CV]  learning_rate=0.02, max_depth=7, min_samples_leaf=20, min_samples_split=5, n_estimators=3000, total=  57.9s
[CV] learning_rate=0.02, max_depth=7, min_samples_leaf=20, min_samples_split=10, n_estimators=4000 
[CV]  learning_rate=0.02, max_depth=7, min_samples_leaf=20, min_samples_split=5, n_estimators=3000, total=  58.2s
[CV] learning_rate=0.02, max_d

[CV] learning_rate=0.02, max_depth=7, min_samples_leaf=25, min_samples_split=5, n_estimators=3000 
[CV]  learning_rate=0.02, max_depth=7, min_samples_leaf=25, min_samples_split=5, n_estimators=4000, total= 1.2min
[CV] learning_rate=0.02, max_depth=7, min_samples_leaf=25, min_samples_split=5, n_estimators=3000 
[CV]  learning_rate=0.02, max_depth=7, min_samples_leaf=25, min_samples_split=5, n_estimators=5000, total= 1.4min
[CV] learning_rate=0.02, max_depth=7, min_samples_leaf=25, min_samples_split=5, n_estimators=3000 
[CV]  learning_rate=0.02, max_depth=7, min_samples_leaf=25, min_samples_split=5, n_estimators=5000, total= 1.5min
[CV] learning_rate=0.02, max_depth=7, min_samples_leaf=25, min_samples_split=5, n_estimators=3000 
[CV]  learning_rate=0.02, max_depth=7, min_samples_leaf=25, min_samples_split=5, n_estimators=4000, total= 1.2min
[CV] learning_rate=0.02, max_depth=7, min_samples_leaf=25, min_samples_split=5, n_estimators=3000 
[CV]  learning_rate=0.02, max_depth=7, min_sample

[CV]  learning_rate=0.02, max_depth=7, min_samples_leaf=25, min_samples_split=15, n_estimators=4000, total=  56.3s


[Parallel(n_jobs=8)]: Done 4500 out of 4500 | elapsed: 542.6min finished


{'learning_rate': 0.02, 'max_depth': 4, 'min_samples_leaf': 12, 'min_samples_split': 5, 'n_estimators': 3000}
R2 on train data: 0.9739711429198467
RMSE on train data: 0.06260594162911186
R2 on test data: 0.9008417454636115
RMSE on test data: 0.12079644350609321


In [ ]:
xgb_reg_model = xgb.XGBRegressor(
                     colsample_bytree=0.2,
                     gamma=0.0,
                     learning_rate=0.01,
                     max_depth=4,
                     min_child_weight=1.5,
                     n_estimators=8000,                                                                  
                     reg_alpha=0.9,
                     reg_lambda=0.6,
                     subsample=0.2,
                     seed=42,
                     silent=1,
                    nthread=-1)

xgb_reg_model.fit(scaled_train_X, trainY)

y_pred_train = xgb_reg_model.predict(scaled_train_X)
y_pred_test = xgb_reg_model.predict(scaled_test_X)


print('RMSE on train data: {}'.format(np.sqrt(mean_squared_error(y_pred_train, trainY))))
print('R2 of train data: {}'.format(r2_score(y_pred_train, trainY)))

print('RMSE on test data: {}'.format(np.sqrt(mean_squared_error(y_pred_test, testY))))
print('R2 of test data: {}'.format(r2_score(y_pred_test, testY)))

RMSE on train data: 0.07306357037578955
R2 of train data: 0.9669067031044456
RMSE on test data: 0.11487305484970117
R2 of test data: 0.9090299233738116
